In [69]:
import pandas as pd
import re
from geopy.geocoders import Nominatim

# Cleaning the 'races' file

In [70]:
races = pd.read_csv('../data/races.csv', index_col = 0)

In [71]:
races.head(10)

,0,1,0.1,1.1,2
0,19,NOV,Valencia MotoGP™ Official Test,Circuit Ricardo Tormo,SPAIN
1,25,NOV,Jerez MotoGP™ Official Test,Circuito de Jerez - Angel Nieto,SPAIN
2,7,FEB,Sepang MotoGP™ Official Test,Sepang International Circuit,MALAYSIA
3,19,FEB,Jerez Moto2™-Moto3™ Test,Circuito de Jerez - Angel Nieto,SPAIN
4,22,FEB,Qatar MotoGP™ Test,Losail International Circuit,QATAR
5,28,FEB,Qatar Moto2™-Moto3™ Test,Losail International Circuit,QATAR
6,8,MAR,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
7,22,MAR,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
8,5,APR,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
9,19,APR,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA


## Renaming columns

In [72]:
races.rename(columns = {'0':'Day', '1':'Month', '0.1':'Race', '1.1':'Circuit', '2':'Country'}, inplace = True)

In [73]:
races.head()

,Day,Month,Race,Circuit,Country
0,19,NOV,Valencia MotoGP™ Official Test,Circuit Ricardo Tormo,SPAIN
1,25,NOV,Jerez MotoGP™ Official Test,Circuito de Jerez - Angel Nieto,SPAIN
2,7,FEB,Sepang MotoGP™ Official Test,Sepang International Circuit,MALAYSIA
3,19,FEB,Jerez Moto2™-Moto3™ Test,Circuito de Jerez - Angel Nieto,SPAIN
4,22,FEB,Qatar MotoGP™ Test,Losail International Circuit,QATAR


## Removing 'Tests'

For this project, I will focus only in scoring races which are called 'Grand Prix'.

In [74]:
# The new dataframe should not include races with 'Test' word in the 'Race' column.
races = races.loc[races['Race'].str.contains('Test') == False]

In [75]:
races.head()

,Day,Month,Race,Circuit,Country
6,8,MAR,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
7,22,MAR,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
8,5,APR,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
9,19,APR,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
10,3,MAY,5 - Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


## Updating dates

Now we removed the tests, all the dates are for same year 2020. I will create a new column with 'Date' as a DateTime

In [76]:
races['Date'] = pd.to_datetime((races['Day'].astype(str)+races['Month'] + '2020'), errors = 'coerce', infer_datetime_format=True)

In [77]:
races = races[['Date', 'Race', 'Circuit', 'Country']].reset_index(drop=True)

In [78]:
races.head()

,Date,Race,Circuit,Country
0,2020-03-08,1 - Grand Prix of Qatar,Losail International Circuit,QATAR
1,2020-03-22,2 - OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
2,2020-04-05,3 - Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
3,2020-04-19,4 - Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
4,2020-05-03,5 - Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


In [79]:
races.dtypes

Date       datetime64[ns]
Race               object
Circuit            object
Country            object
dtype: object

## Removing numbers on Race names

As they are already ordered, we can remove the numbers at the beginning of the race's name.

In [80]:
races['Race'] = races['Race'].apply(lambda x: re.sub(r'\d+ - ','',x))

In [81]:
races.head()

,Date,Race,Circuit,Country
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR
1,2020-03-22,OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA
4,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez - Angel Nieto,SPAIN


## Checking duplicates

Except for the country, the other columns should not have any duplicates.

In [82]:
races['Date'].duplicated(keep = False).any()

False

In [83]:
races['Race'].duplicated(keep = False).any()

False

In [84]:
races['Circuit'].duplicated(keep = False).any()

True

In [85]:
races.loc[races['Circuit'].duplicated(keep = False)]

,Date,Race,Circuit,Country
1,2020-03-22,OR Thailand Grand Prix,TT Circuit Assen,NETHERLANDS
9,2020-06-28,Motul TT Assen,TT Circuit Assen,NETHERLANDS


The calendar from motogp.com has an error, and so the data scraped. As is just one mistake (circuit and country for Thailand Grand Prix) I will correct it manually

In [86]:
races.at[1,'Circuit'] = 'Chang International Circuit'

In [87]:
races.at[1,'Country'] = 'THAILAND'

In [88]:
races.loc[races['Circuit'].duplicated(keep = False)]

,Date,Race,Circuit,Country


## Looking for coordinates with GeoPy

In [89]:
# creating a geolocator to use it to get coordinates
geolocator = Nominatim()

# checking the geolocator
races['Circuit'].apply(geolocator.geocode)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


0     (Lusail International Circuit, أم صلال, ‏قطر, ...
1     (Sepang International Circuit, Jalan Kuarters ...
2     (Circuit of The Americas, Larkdale Lane, Lake ...
3     (Termas de Río Hondo, Departamento Río Hondo, ...
4                                                  None
5     (Le Mans, Sarthe, Pays de la Loire, France mét...
6     (Autodromo del Mugello, Mugellino, Omo morto, ...
7     (Circuit de Barcelona-Catalunya, BV-5003, Mont...
8     (Sachsenring, Marienthal Ost, Zwickau-West, Zw...
9     (TT Circuit Assen, TT-tunnelweg, Assen, Drenth...
10    (KymiRing, 748, Kymentie, Iitti, Kouvolan seut...
11    (Automotodrom (rozc.), 3842, Žebětín, Brno, ok...
12    (Red Bull Ring, Spielberg, Murtal, Steiermark,...
13    (Silverstone Circuit, Village Corner, Silverst...
14    (Misano World Circuit Marco Simoncelli, Via Ca...
15    (Motorland Aragón, Puigmoreno, Alcañiz, Bajo A...
16    (ツインリンクもてぎ, Twin Ring Motegi, 茂木町, 芳賀郡, 栃木県, 関...
17    (Phillip Island, Norfolk Island, Australia

As we can see above, the geolocator couldn't find the direction for row 4. Let's check the name.

In [90]:
races.loc[4, 'Circuit']

'Circuito de Jerez - Angel Nieto'

The problem may be due to the append 'Angel Nieto' to the name of the circuit. Let's remove it:

In [91]:
races.at[4, 'Circuit'] = 'Circuito de Jerez'

In [92]:
races.loc[4, 'Circuit']

'Circuito de Jerez'

Now I can try again to get the directions. But this time I will get directly the coordinates

In [93]:
races['Latitude'] = races['Circuit'].apply(geolocator.geocode).apply(lambda x: x.latitude)

In [95]:
races['Longitude'] = races['Circuit'].apply(geolocator.geocode).apply(lambda x: x.longitude)

In [96]:
races.head()

,Date,Race,Circuit,Country,Latitude,Longitude
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR,25.491000,51.452068
1,2020-03-22,OR Thailand Grand Prix,Chang International Circuit,THAILAND,2.760191,101.736859
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES,30.138715,-97.636410
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA,-27.495925,-64.864078
4,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez,SPAIN,36.694447,-6.156317


In [97]:
races.dtypes

Date         datetime64[ns]
Race                 object
Circuit              object
Country              object
Latitude            float64
Longitude           float64
dtype: object

## Races dataframe cleaned

In [98]:
races.to_csv('../data/races_cleaned.csv')

In [100]:
# to plot the circuits in Tableau Public, is better to have an excel file
races.to_excel('../data/races_cleaned.xlsx')

# Cleaning the 'riders' file

In [157]:
riders = pd.read_csv('../data/riders.csv', index_col = 0)

In [158]:
riders.head(10)

,0,1,2,3
0,Andrea Dovizioso,Ducati Team,Bike: Ducati,Forlimpopoli
1,Johann Zarco,Reale Avintia Racing,Bike: Ducati,Cannes
2,Danilo Petrucci,Ducati Team,Bike: Ducati,Terni
3,Maverick Viñales,Monster Energy Yamaha MotoGP,Bike: Yamaha,Figueres
4,Fabio Quartararo,Petronas Yamaha SRT,Bike: Yamaha,Nice
5,Franco Morbidelli,Petronas Yamaha SRT,Bike: Yamaha,Roma
6,Iker Lecuona,Red Bull KTM Tech 3,Bike: KTM,Valencia
7,Andrea Iannone,Aprilia Racing Team Gresini,Bike: Aprilia,Vasto
8,Takaaki Nakagami,LCR Honda IDEMITSU,Bike: KTM,Chiba
9,Brad Binder,Red Bull KTM Factory Racing,Bike: KTM,Potchefstroom


In [164]:
riders.isnull().sum()

Name          0
Team          0
Bike          0
BirthPlace    0
dtype: int64

## Renaming columns

In [159]:
riders.rename(columns = {'0':'Name', '1':'Team', '2':'Bike', '3':'BirthPlace'}, inplace = True)

In [160]:
riders.head()

,Name,Team,Bike,BirthPlace
0,Andrea Dovizioso,Ducati Team,Bike: Ducati,Forlimpopoli
1,Johann Zarco,Reale Avintia Racing,Bike: Ducati,Cannes
2,Danilo Petrucci,Ducati Team,Bike: Ducati,Terni
3,Maverick Viñales,Monster Energy Yamaha MotoGP,Bike: Yamaha,Figueres
4,Fabio Quartararo,Petronas Yamaha SRT,Bike: Yamaha,Nice


## Removing 'Bike:' text in Bike column

The word 'Bike:' in the bike column is redundant information. Let's remove it.

In [161]:
riders['Bike'] = riders['Bike'].apply(lambda x: re.sub(r'Bike: ','',x))

In [162]:
riders.head()

,Name,Team,Bike,BirthPlace
0,Andrea Dovizioso,Ducati Team,Ducati,Forlimpopoli
1,Johann Zarco,Reale Avintia Racing,Ducati,Cannes
2,Danilo Petrucci,Ducati Team,Ducati,Terni
3,Maverick Viñales,Monster Energy Yamaha MotoGP,Yamaha,Figueres
4,Fabio Quartararo,Petronas Yamaha SRT,Yamaha,Nice


## Checking duplicates

We can't have more than one rider with the same name.

In [165]:
riders['Name'].duplicated(keep = False).any()

False

## Checking errors in Teams and Bikes

As they can be repeated, let's check if all of them are written properly

In [182]:
riders['Team'].value_counts()

Red Bull KTM Factory Racing     3
Red Bull KTM Tech 3             2
LCR Honda                       2
Team SUZUKI ECSTAR              2
Reale Avintia Racing            2
Monster Energy Yamaha MotoGP    2
Aprilia Racing Team Gresini     2
Repsol Honda Team               2
Ducati Team                     2
Petronas Yamaha SRT             2
Pramac Racing                   2
Name: Team, dtype: int64

Although we have two different Teams for LCR Honda, due to main sponsors, both are the same team structure, so we will consider them as a unique Team

In [168]:
riders.loc[riders['Team'].str.contains('LCR')]

,Name,Team,Bike,BirthPlace
8,Takaaki Nakagami,LCR Honda IDEMITSU,KTM,Chiba
10,Cal Crutchlow,LCR Honda CASTROL,Honda,Coventry


Another error has raised: Honda Team with KTM Bike is not possible. After fixing the Team, I will look for the same bike in Team and Bike

In [175]:
riders.at[8,'Team'] = 'LCR Honda'

In [176]:
riders.at[10,'Team']= 'LCR Honda'

In [177]:
riders['Team'].value_counts()

Red Bull KTM Factory Racing     3
Red Bull KTM Tech 3             2
LCR Honda                       2
Team SUZUKI ECSTAR              2
Reale Avintia Racing            2
Monster Energy Yamaha MotoGP    2
Aprilia Racing Team Gresini     2
Repsol Honda Team               2
Ducati Team                     2
Petronas Yamaha SRT             2
Pramac Racing                   2
Name: Team, dtype: int64

Checking if there is any other error between the name of the team and the bike. At least we have 1 in row[8]

In [180]:
riders.loc[riders.apply(lambda x: x['Bike'] in x['Team'], axis = 1) == False]

,Name,Team,Bike,BirthPlace
1,Johann Zarco,Reale Avintia Racing,Ducati,Cannes
8,Takaaki Nakagami,LCR Honda,KTM,Chiba
11,Joan Mir,Team SUZUKI ECSTAR,Suzuki,Palma de Mallorca
13,Alex Rins,Team SUZUKI ECSTAR,Suzuki,Barcelona
14,Jack Miller,Pramac Racing,Ducati,Townsville
17,Tito Rabat,Reale Avintia Racing,Ducati,Barcelona
18,Francesco Bagnaia,Pramac Racing,Ducati,Torino


From the list above we can see that there are two Ducati Teams without the name of the bike in the name of the team: Reale Avintia Racing and Pramac Racing. But this is not an error.

Also, the Team Suzuki appears in the list due to capital letters, but this is not an error.

The unique error is the one it was raised before, having a KTM bike in the LCR Honda team, and this needs to be fixed.

In [181]:
riders.at[8,'Bike'] = 'Honda'

## Looking for coordinates with GeoPy

For this project I will consider the Birth Place as the riders' home. Although it may not be exactly the same place, is enough to get an approximate idea of the distance travelled during the whole season.

In [183]:
riders['Lat_Lon'] = riders['BirthPlace'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [184]:
riders.head()

,Name,Team,Bike,BirthPlace,Lat_Lon
0,Andrea Dovizioso,Ducati Team,Ducati,Forlimpopoli,"(44.1873049, 12.1259056)"
1,Johann Zarco,Reale Avintia Racing,Ducati,Cannes,"(43.5515198, 7.0134418)"
2,Danilo Petrucci,Ducati Team,Ducati,Terni,"(42.6537515, 12.4398116328367)"
3,Maverick Viñales,Monster Energy Yamaha MotoGP,Yamaha,Figueres,"(42.2666314, 2.9638434)"
4,Fabio Quartararo,Petronas Yamaha SRT,Yamaha,Nice,"(43.7009358, 7.2683912)"


## Riders dataframe cleaned

In [185]:
riders.to_csv('../data/riders_cleaned.csv')